In [22]:
import numpy as np

class RecurrentNeuralNetwork:
    #input (in t), expected output (shifted by one time step), num of words (num of recurrences), array of expected outputs, learning rate
    
    def __init__ (self, input, output, recurrences, expected_output, learning_rate):
        #initial input 
        self.x = np.zeros(input)
        #input size 
        self.input = input
        #expected output (shifted by one time step)
        self.y = np.zeros(output)
        #output size
        self.output = output
        #weight matrix for interpreting results from hidden state cell (num words x num words matrix)
        #random initialization is crucial here
        self.w = np.random.random((output, output))
        #matrix used in RMSprop in order to decay the learning rate
        self.G = np.zeros_like(self.w)
        #length of the recurrent network - number of recurrences i.e num of inputs
        self.recurrences = recurrences
        #learning rate 
        self.learning_rate = learning_rate
        #array for storing inputs
        self.ia = np.zeros((recurrences+1,input))
        #array for storing cell states
        self.ca = np.zeros((recurrences+1,output))
        #array for storing outputs
        self.oa = np.zeros((recurrences+1,output))
        #array for storing hidden states
        self.ha = np.zeros((recurrences+1,output))
        #forget gate 
        self.af = np.zeros((recurrences+1,output))
        #input gate
        self.ai = np.zeros((recurrences+1,output))
        #cell state
        self.ac = np.zeros((recurrences+1,output))
        #output gate
        self.ao = np.zeros((recurrences+1,output))
        #array of expected output values
        self.expected_output = np.vstack((np.zeros(expected_output.shape[0]), expected_output.T))
        #declare LSTM cell (input, output, amount of recurrence, learning rate)
        self.LSTM = LSTM(input, output, recurrences, learning_rate)
    
    #activation function. simple nonlinearity, converts influx floats into probabilities between 0 and 1
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    #the derivative of the sigmoid function. We need it to compute gradients for backpropagation
    def dsigmoid(self, x):
        return self.sigmoid(x) * (1 - self.sigmoid(x))    
    
    #Here is where magic happens: We apply a series of matrix operations to our input and compute an output 
    def forwardProp(self):
        for i in range(1, self.recurrences+1):
            self.LSTM.x = np.hstack((self.ha[i-1], self.x))
            cs, hs, f, inp, c, o = self.LSTM.forwardProp()
            #store cell state from the forward propagation
            self.ca[i] = cs #cell state
            self.ha[i] = hs #hidden state
            self.af[i] = f #forget state
            self.ai[i] = inp #inpute gate
            self.ac[i] = c #cell state
            self.ao[i] = o #output gate
            self.oa[i] = self.sigmoid(np.dot(self.w, hs)) #activate the weight*input
            self.x = self.expected_output[i-1]
        return self.oa
   
    
    def backProp(self):
        #Backpropagation of our weight matrices (Both in our Recurrent network + weight matrices inside LSTM cell)
        #start with an empty error value 
        totalError = 0
        #initialize matrices for gradient updates
        #First, these are RNN level gradients
        #cell state
        dfcs = np.zeros(self.output)
        #hidden state,
        dfhs = np.zeros(self.output)
        #weight matrix
        tu = np.zeros((self.output,self.output))
        #Next, these are LSTM level gradients
        #forget gate
        tfu = np.zeros((self.output, self.input+self.output))
        #input gate
        tiu = np.zeros((self.output, self.input+self.output))
        #cell unit
        tcu = np.zeros((self.output, self.input+self.output))
        #output gate
        tou = np.zeros((self.output, self.input+self.output))
        #loop backwards through recurrences
        for i in range(self.recurrences, -1, -1):
            #error = calculatedOutput - expectedOutput
            error = self.oa[i] - self.expected_output[i]
            #calculate update for weight matrix
            #Compute the partial derivative with (error * derivative of the output) * hidden state
            tu += np.dot(np.atleast_2d(error * self.dsigmoid(self.oa[i])), np.atleast_2d(self.ha[i]).T)
            #Now propagate the error back to exit of LSTM cell
            #1. error * RNN weight matrix
            error = np.dot(error, self.w)
            #2. set input values of LSTM cell for recurrence i (horizontal stack of array output, hidden + input)
            self.LSTM.x = np.hstack((self.ha[i-1], self.ia[i]))
            #3. set cell state of LSTM cell for recurrence i (pre-updates)
            self.LSTM.cs = self.ca[i]
            #Finally, call the LSTM cell's backprop and retreive gradient updates
            #Compute the gradient updates for forget, input, cell unit, and output gates + cell states + hiddens states
            fu, iu, cu, ou, dfcs, dfhs = self.LSTM.backProp(error, self.ca[i-1], self.af[i], self.ai[i], self.ac[i], self.ao[i], dfcs, dfhs)
            #Accumulate the gradients by calculating total error (not necesarry, used to measure training progress)
            totalError += np.sum(error)
            #forget gate
            tfu += fu
            #input gate
            tiu += iu
            #cell state
            tcu += cu
            #output gate
            tou += ou
        #update LSTM matrices with average of accumulated gradient updates    
        self.LSTM.update(tfu/self.recurrences, tiu/self.recurrences, tcu/self.recurrences, tou/self.recurrences) 
        #update weight matrix with average of accumulated gradient updates  
        self.update(tu/self.recurrences)
        #return total error of this iteration
        return totalError
    
    def update(self, u):
        #Implementation of RMSprop in the vanilla world
        #We decay our learning rate to increase convergence speed
        self.G = 0.95 * self.G + 0.1 * u**2  
        self.w -= self.learning_rate/np.sqrt(self.G + 1e-8) * u
        return
    
    #We define a sample function which produces the output once we trained our model
    #let's say that we feed an input observed at time t, our model will produce an output that can be 
    #observe in time t+1 
    def sample(self):
         #loop through recurrences - start at 1 so the 0th entry of all output will be an array of 0's
        for i in range(1, self.recurrences+1):
            #set input for LSTM cell, combination of input (previous output) and previous hidden state
            self.LSTM.x = np.hstack((self.ha[i-1], self.x))
            #run forward prop on the LSTM cell, retrieve cell state and hidden state
            cs, hs, f, inp, c, o = self.LSTM.forwardProp()
            #store input as vector
            maxI = np.argmax(self.x)
            self.x = np.zeros_like(self.x)
            self.x[maxI] = 1
            self.ia[i] = self.x #Use np.argmax?
            #store cell states
            self.ca[i] = cs
            #store hidden state
            self.ha[i] = hs
            #forget gate
            self.af[i] = f
            #input gate
            self.ai[i] = inp
            #cell state
            self.ac[i] = c
            #output gate
            self.ao[i] = o
            #calculate output by multiplying hidden state with weight matrix
            self.oa[i] = self.sigmoid(np.dot(self.w, hs))
            #compute new input
            maxI = np.argmax(self.oa[i])
            newX = np.zeros_like(self.x)
            newX[maxI] = 1
            self.x = newX
        #return all outputs    
        return self.oa

In [23]:
import numpy as np
class LSTM:
    # LSTM cell (input, output, amount of recurrence, learning rate)
    def __init__ (self, input, output, recurrences, learning_rate):
        #input size
        self.x = np.zeros(input+output)
        #input size
        self.input = input + output
        #output 
        self.y = np.zeros(output)
        #output size
        self.output = output
        #cell state intialized as size of prediction
        self.cs = np.zeros(output)
        #how often to perform recurrence
        self.recurrences = recurrences
        #balance the rate of training (learning rate)
        self.learning_rate = learning_rate
        #init weight matrices for our gates
        #forget gate
        self.f = np.random.random((output, input+output))
        #input gate
        self.i = np.random.random((output, input+output))
        #cell state
        self.c = np.random.random((output, input+output))
        #output gate
        self.o = np.random.random((output, input+output))
        #forget gate gradient
        self.Gf = np.zeros_like(self.f)
        #input gate gradient
        self.Gi = np.zeros_like(self.i)
        #cell state gradient
        self.Gc = np.zeros_like(self.c)
        #output gate gradient
        self.Go = np.zeros_like(self.o)
    
    #activation function. simple nonlinearity, converts influx floats into probabilities between 0 and 1
    #same as in rnn class
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    #The derivativeative of the sigmoid function. We need it to compute gradients for backpropagation
    def dsigmoid(self, x):
        return self.sigmoid(x) * (1 - self.sigmoid(x))
    
    #Within the LSTM cell we are going to use a tanh activation function.
    #Long story short: This leads to stronger gradients (derivativeatives are higher) which is usefull as our data is centered around 0, 
    def tangent(self, x):
        return np.tanh(x)
    
    #derivativeative for computing gradients
    def dtangent(self, x):
        return 1 - np.tanh(x)**2
    
    #Here is where magic happens: We apply a series of matrix operations to our input and compute an output 
    def forwardProp(self):
        f = self.sigmoid(np.dot(self.f, self.x))
        self.cs *= f
        i = self.sigmoid(np.dot(self.i, self.x))
        c = self.tangent(np.dot(self.c, self.x))
        self.cs += i * c
        o = self.sigmoid(np.dot(self.o, self.x))
        self.y = o * self.tangent(self.cs)
        return self.cs, self.y, f, i, c, o
    
   
    def backProp(self, e, pcs, f, i, c, o, dfcs, dfhs):
        #error = error + hidden state derivativeative. we clip the value between -6 and 6 to prevent vanishing
        e = np.clip(e + dfhs, -6, 6)
        #multiply error by activated cell state to compute output derivativeative
        do = self.tangent(self.cs) * e
        #output update = (output derivative * activated output) * input
        ou = np.dot(np.atleast_2d(do * self.dtangent(o)).T, np.atleast_2d(self.x))
        #derivativeative of cell state = error * output * derivativeative of cell state + derivative cell
        #compute gradients and update them in reverse order!
        dcs = np.clip(e * o * self.dtangent(self.cs) + dfcs, -6, 6)
        #derivativeative of cell = derivativeative cell state * input
        dc = dcs * i
        #cell update = derivativeative cell * activated cell * input
        cu = np.dot(np.atleast_2d(dc * self.dtangent(c)).T, np.atleast_2d(self.x))
        #derivativeative of input = derivative cell state * cell
        di = dcs * c
        #input update = (derivative input * activated input) * input
        iu = np.dot(np.atleast_2d(di * self.dsigmoid(i)).T, np.atleast_2d(self.x))
        #derivative forget = derivative cell state * all cell states
        df = dcs * pcs
        #forget update = (derivative forget * derivative forget) * input
        fu = np.dot(np.atleast_2d(df * self.dsigmoid(f)).T, np.atleast_2d(self.x))
        #derivative cell state = derivative cell state * forget
        dpcs = dcs * f
        #derivative hidden state = (derivative cell * cell) * output + derivative output * output * output derivative input * input * output + derivative forget
        #* forget * output
        dphs = np.dot(dc, self.c)[:self.output] + np.dot(do, self.o)[:self.output] + np.dot(di, self.i)[:self.output] + np.dot(df, self.f)[:self.output] 
        #return update gradients for forget, input, cell, output, cell state, hidden state
        return fu, iu, cu, ou, dpcs, dphs
            
    def update(self, fu, iu, cu, ou):
        #Update forget, input, cell, and output gradients
        self.Gf = 0.9 * self.Gf + 0.1 * fu**2 
        self.Gi = 0.9 * self.Gi + 0.1 * iu**2   
        self.Gc = 0.9 * self.Gc + 0.1 * cu**2   
        self.Go = 0.9 * self.Go + 0.1 * ou**2   
        
        #Update our gates using our gradients
        self.f -= self.learning_rate/np.sqrt(self.Gf + 1e-8) * fu
        self.i -= self.learning_rate/np.sqrt(self.Gi + 1e-8) * iu
        self.c -= self.learning_rate/np.sqrt(self.Gc + 1e-8) * cu
        self.o -= self.learning_rate/np.sqrt(self.Go + 1e-8) * ou
        return

In [24]:
def read_text_file():
    #Open textfile and return a separated input and output variable(series of words)
    with open("main.txt", "r") as text_file:
        data = text_file.read()
    text = list(data)
    outputSize = len(text)
    data = list(set(text))
    uniqueWords, dataSize = len(data), len(data) 
    returnData = np.zeros((uniqueWords, dataSize))
    for i in range(0, dataSize):
        returnData[i][i] = 1
    returnData = np.append(returnData, np.atleast_2d(data), axis=0)
    output = np.zeros((uniqueWords, outputSize))
    for i in range(0, outputSize):
        index = np.where(np.asarray(data) == text[i])
        output[:,i] = returnData[0:-1,index[0]].astype(float).ravel()  
    return returnData, uniqueWords, output, outputSize, data

#Write the predicted output (series of words) to disk
def export_to_textfile(output, data):
    finalOutput = np.zeros_like(output)
    prob = np.zeros_like(output[0])
    outputText = ""
    print(len(data))
    print(output.shape[0])
    for i in range(0, output.shape[0]):
        for j in range(0, output.shape[1]):
            prob[j] = output[i][j] / np.sum(output[i])
        outputText += np.random.choice(data, p=prob)    
    with open("output.txt", "w") as text_file:
        text_file.write(outputText)
    return

In [25]:
print("Initialize Hyperparameters")
iterations = 10000
learningRate = 0.001
print("Reading Inputa/Output data from disk")
returnData, numCategories, expectedOutput, outputSize, data = read_text_file()
print("Reading from disk done. Proceeding with training.")
#Initialize the RNN using our hyperparameters and data
RNN = RecurrentNeuralNetwork(numCategories, numCategories, outputSize, expectedOutput, learningRate)

#training time!
for i in range(1, iterations):
    #Predict the next word of each word
    RNN.forwardProp()
    #update all our weights using our error
    error = RNN.backProp()
    #For a given error threshold
    print("Reporting error on iteration ", i, ": ", error)
    if error > -10 and error < 10 or i % 10 == 0:
        #We provide a seed word
        seed = np.zeros_like(RNN.x)
        maxI = np.argmax(np.random.random(RNN.x.shape))
        seed[maxI] = 1
        RNN.x = seed  
        #and predict the upcoming one
        output = RNN.sample()
        print(output)    
        #finally, we store it to disk
        export_to_textfile(output, data)
        print("Done Writing")
print("Train/Prediction routine complete.")

Initialize Hyperparameters
Reading Inputa/Output data from disk
Reading from disk done. Proceeding with training.
Reporting error on iteration  1 :  406230.8164428437
Reporting error on iteration  2 :  404314.9429615515
Reporting error on iteration  3 :  402507.5316385683
Reporting error on iteration  4 :  400980.8806115999
Reporting error on iteration  5 :  399642.8350705333
Reporting error on iteration  6 :  398431.5398290793
Reporting error on iteration  7 :  397312.3942318209
Reporting error on iteration  8 :  396263.7790017528
Reporting error on iteration  9 :  395271.14234280237
Reporting error on iteration  10 :  394324.1497161591
[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.99996941 0.99998067 0.99997241 ... 0.99993831 0.99997154 0.99991488]
 [0.99999995 0.99999998 0.99999997 ... 0.99999989 0.99999998 0.99999976]
 ...
 [0.99999995 0.99999998 0.99999997 ... 0.99999989 0.99999998 0.99999976]
 [0.99999995 0.99999998 0.99999997 ... 0.99999989 0.99999

D:\adiarun\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in double_scalars
D:\adiarun\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in less


Done Writing
Reporting error on iteration  11 :  393415.1563514846
Reporting error on iteration  12 :  392538.34628274664
Reporting error on iteration  13 :  391689.1123859197
Reporting error on iteration  14 :  390863.81741081085
Reporting error on iteration  15 :  390059.485205145
Reporting error on iteration  16 :  389273.65661236265
Reporting error on iteration  17 :  388504.273473356
Reporting error on iteration  18 :  387749.59399219597
Reporting error on iteration  19 :  387008.12970047246
Reporting error on iteration  20 :  386278.59765345464
[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.99997506 0.99998403 0.99997592 ... 0.99994122 0.99998462 0.99993313]
 [0.99999994 0.99999998 0.99999996 ... 0.99999985 0.99999997 0.99999966]
 ...
 [0.99999994 0.99999998 0.99999996 ... 0.99999985 0.99999997 0.99999966]
 [0.99999994 0.99999998 0.99999996 ... 0.99999985 0.99999997 0.99999966]
 [0.99999994 0.99999998 0.99999996 ... 0.99999985 0.99999997 0.99999966]]

Reporting error on iteration  94 :  341306.9200268968
Reporting error on iteration  95 :  340729.01827317907
Reporting error on iteration  96 :  340151.23380604974
Reporting error on iteration  97 :  339573.56068062654
Reporting error on iteration  98 :  338995.9932554851
Reporting error on iteration  99 :  338418.52617683547
Reporting error on iteration  100 :  337841.1543635522
[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.99988758 0.99993758 0.9999413  ... 0.99983122 0.99993827 0.99971182]
 [0.99999946 0.99999979 0.99999964 ... 0.99999873 0.99999978 0.99999712]
 ...
 [0.99999946 0.9999998  0.99999964 ... 0.99999873 0.99999978 0.99999712]
 [0.99999946 0.9999998  0.99999964 ... 0.99999873 0.99999978 0.99999712]
 [0.99999946 0.9999998  0.99999964 ... 0.99999873 0.99999978 0.99999712]]
36
647
Done Writing
Reporting error on iteration  101 :  337263.83906002826
Reporting error on iteration  102 :  336686.70814573666
Reporting error on iteration  103 :  3361

Reporting error on iteration  176 :  294063.4586515579
Reporting error on iteration  177 :  293487.810597157
Reporting error on iteration  178 :  292912.16336294624
Reporting error on iteration  179 :  292336.5168288997
Reporting error on iteration  180 :  291760.8708794247
[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.99963529 0.99976603 0.99968274 ... 0.99931369 0.99975768 0.99891046]
 [0.99999578 0.99999841 0.99999723 ... 0.99999019 0.99999828 0.99997768]
 ...
 [0.99999583 0.99999843 0.99999727 ... 0.99999027 0.9999983  0.99997789]
 [0.99999583 0.99999843 0.99999727 ... 0.99999027 0.9999983  0.99997789]
 [0.99999583 0.99999843 0.99999727 ... 0.99999027 0.9999983  0.99997789]]
36
647
Done Writing
Reporting error on iteration  181 :  291185.06607468345
Reporting error on iteration  182 :  290609.62771045073
Reporting error on iteration  183 :  290033.9804947529
Reporting error on iteration  184 :  289458.3335622759
Reporting error on iteration  185 :  28

...

Done Writing
Reporting error on iteration  9641 :  685.8892819793114
Reporting error on iteration  9642 :  625.9469397280949
Reporting error on iteration  9643 :  685.8476194043066
Reporting error on iteration  9644 :  625.6654907235571
Reporting error on iteration  9645 :  685.9743676719925
Reporting error on iteration  9646 :  625.569748987888
Reporting error on iteration  9647 :  685.9814324474656
Reporting error on iteration  9648 :  625.5448270375198
Reporting error on iteration  9649 :  685.9483831233673
Reporting error on iteration  9650 :  625.5389899399555
[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.03447721 0.04750966 0.03285826 ... 0.01782874 0.04153112 0.0100725 ]
 [0.00575521 0.03052218 0.01200618 ... 0.00606801 0.07437392 0.0042485 ]
 ...
 [0.00684895 0.02138906 0.00965394 ... 0.00515156 0.07608242 0.00480205]
 [0.00684895 0.02138906 0.00965394 ... 0.00515156 0.07608242 0.00480205]
 [0.00684895 0.02138906 0.00965394 ... 0.00515156 0.076082

Reporting error on iteration  9721 :  685.0811223933674
Reporting error on iteration  9722 :  624.9206961997171
Reporting error on iteration  9723 :  685.121111563018
Reporting error on iteration  9724 :  624.6334308666906
Reporting error on iteration  9725 :  685.229171124453
Reporting error on iteration  9726 :  624.5659344476657
Reporting error on iteration  9727 :  685.2110409339665
Reporting error on iteration  9728 :  624.5643239038177
Reporting error on iteration  9729 :  685.1628802456066
Reporting error on iteration  9730 :  624.5722220849344
[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.02283194 0.04913573 0.03193677 ... 0.01288374 0.03358902 0.00748321]
 [0.00565632 0.03251444 0.01277679 ... 0.00610276 0.07967612 0.00467273]
 ...
 [0.00686216 0.02139185 0.00968268 ... 0.00517007 0.07612311 0.00481322]
 [0.00686216 0.02139185 0.00968268 ... 0.00517007 0.07612311 0.00481322]
 [0.00686216 0.02139185 0.00968268 ... 0.00517007 0.07612311 0.00481322]

Reporting error on iteration  9802 :  686.0405348158525
Reporting error on iteration  9803 :  625.1936881242166
Reporting error on iteration  9804 :  685.7353742239262
Reporting error on iteration  9805 :  625.3447632774357
Reporting error on iteration  9806 :  685.6260300109244
Reporting error on iteration  9807 :  625.3646311990296
Reporting error on iteration  9808 :  685.601711954132
Reporting error on iteration  9809 :  625.3351199060654
Reporting error on iteration  9810 :  685.6043456568171
[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.02393681 0.03696415 0.0277718  ... 0.0140132  0.03645443 0.006862  ]
 [0.0057609  0.03044188 0.01210928 ... 0.00602635 0.07496416 0.00423606]
 ...
 [0.00704617 0.02192031 0.00995274 ... 0.00531799 0.07789935 0.00494438]
 [0.00704617 0.02192031 0.00995274 ... 0.00531799 0.07789935 0.00494438]
 [0.00704617 0.02192031 0.00995274 ... 0.00531799 0.07789935 0.00494438]]
36
647
Done Writing
Reporting error on iteration  981

Reporting error on iteration  9883 :  624.4269065431547
Reporting error on iteration  9884 :  685.2337318223847
Reporting error on iteration  9885 :  624.5774596615267
Reporting error on iteration  9886 :  685.1220994542867
Reporting error on iteration  9887 :  624.6182515067427
Reporting error on iteration  9888 :  685.0877201439123
Reporting error on iteration  9889 :  624.6118243805369
Reporting error on iteration  9890 :  685.0837820752224
[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.02676991 0.04322288 0.04009619 ... 0.01572757 0.0464464  0.01161398]
 [0.00619531 0.03183031 0.01332093 ... 0.00612    0.08070037 0.00509347]
 ...
 [0.00706046 0.0219292  0.00998409 ... 0.00533736 0.07793316 0.00495714]
 [0.00706046 0.0219292  0.00998409 ... 0.00533736 0.07793316 0.00495714]
 [0.00706046 0.0219292  0.00998409 ... 0.00533736 0.07793316 0.00495714]]
36
647
Done Writing
Reporting error on iteration  9891 :  624.4646803468418
Reporting error on iteration  98

Reporting error on iteration  9964 :  625.428456265727
Reporting error on iteration  9965 :  686.0196848492615
Reporting error on iteration  9966 :  625.3644800043064
Reporting error on iteration  9967 :  686.0199037516827
Reporting error on iteration  9968 :  625.3670497034718
Reporting error on iteration  9969 :  685.9875174480317
Reporting error on iteration  9970 :  625.3813953237608
[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.02306252 0.04094712 0.03081468 ... 0.0141153  0.03612071 0.00918036]
 [0.00565759 0.02983453 0.01153182 ... 0.00593093 0.07343674 0.00417195]
 ...
 [0.00685019 0.02155395 0.00972624 ... 0.0052154  0.07590825 0.00479555]
 [0.00685019 0.02155395 0.00972624 ... 0.0052154  0.07590825 0.00479555]
 [0.00685019 0.02155395 0.00972624 ... 0.0052154  0.07590825 0.00479555]]
36
647
Done Writing
Reporting error on iteration  9971 :  685.8401703125045
Reporting error on iteration  9972 :  625.6048613122285
Reporting error on iteration  997